In [4]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

In [5]:
#fetching dataset
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ltXPv2zr5W0S0VrByKWNHN0YZe_XHA6G' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ltXPv2zr5W0S0VrByKWNHN0YZe_XHA6G" -O test.csv && rm -rf /tmp/cookies.txt

--2023-12-15 13:23:57--  https://docs.google.com/uc?export=download&confirm=&id=1ltXPv2zr5W0S0VrByKWNHN0YZe_XHA6G
Resolving docs.google.com (docs.google.com)... 142.251.2.138, 142.251.2.100, 142.251.2.113, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qe17mf6cpfk5q5fg2tdv8vqpjtdi3qn2/1702646625000/07653173915231569853/*/1ltXPv2zr5W0S0VrByKWNHN0YZe_XHA6G?e=download&uuid=f5689781-6049-4df2-a1db-611e542f70ad [following]
--2023-12-15 13:23:58--  https://doc-0o-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qe17mf6cpfk5q5fg2tdv8vqpjtdi3qn2/1702646625000/07653173915231569853/*/1ltXPv2zr5W0S0VrByKWNHN0YZe_XHA6G?e=download&uuid=f5689781-6049-4df2-a1db-611e542f70ad
Resolving doc-0o-3g-docs.googleusercontent.com (doc-0o-3g-docs.googleusercontent.com)... 142.251.2.132, 2607:f

In [6]:
df = pd.read_csv('test.csv')

In [7]:
df.head()

,Unnamed: 0,id,date,likes,username,media,inferred company,hour,month,day of week,Year,Link,Media Type,image_path,image_caption
0,0,1,2019-05-10 11:44:04,2,UPMCnews,[Photo(previewUrl='https://pbs.twimg.com/media...,upmc,11,5,Friday,2019,https://pbs.twimg.com/media/D6NAaxjWwAApBIC?fo...,Photo,0.jpg,arafed bee on a purple flower with yellow pollen
1,1,2,2019-09-03 14:00:01,1,rbcgamnews,[Photo(previewUrl='https://pbs.twimg.com/media...,rbc,14,9,Tuesday,2019,https://pbs.twimg.com/media/EDi39MSWkAEi-a9?fo...,Photo,1.jpg,a white and blue brochure with a green grass a...
2,2,3,2018-12-02 05:51:16,502,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc,5,12,Sunday,2018,https://pbs.twimg.com/media/DtY64HyX4AEoG0v?fo...,Photo,2.jpg,a yellow and white sign that says breaking
3,3,4,2018-08-04 20:03:17,151,BBCSport,[Photo(previewUrl='https://pbs.twimg.com/media...,bbc,20,8,Saturday,2018,https://pbs.twimg.com/media/Djx_G6uXsAAKHbb?fo...,Photo,3.jpg,soccer players in action on a soccer field dur...
4,4,5,2019-02-11 04:29:29,362,uclagymnastics,[Photo(previewUrl='https://pbs.twimg.com/media...,ucla,4,2,Monday,2019,https://pbs.twimg.com/media/DzGQrCjU8AEwsN0?fo...,Photo,4.jpg,a group of women in black leopers are hugging ...


In [3]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "MohdRehan001/bloom-7b1-lora-tagger-for-tweet-generation-1100iter"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.5M [00:00<?, ?B/s]

In [9]:
tds = Dataset.from_pandas(df)
data = DatasetDict()
data['test'] = tds

In [10]:
def clean(example):
  if example['image_caption'] is None:
    example['image_caption'] = ""
  return example

In [11]:
data['test'] = data['test'].map(clean)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [12]:
def merge_columns(example): #creating prompts
    example["prediction"] = example["inferred company"] +" with username " + example['username'] +  " posted a tweet at " + str(example['date']) + " with an image that is " + example['image_caption'] + " with number of likes " + str(example['likes'])+ " that reads ->: "
    return example

In [13]:
data['test'] = data['test'].map(merge_columns)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [14]:
data['test'][0]

{'Unnamed: 0': 0,
 'id': 1,
 'date': '2019-05-10 11:44:04',
 'likes': 2,
 'username': 'UPMCnews',
 'media': "[Photo(previewUrl='https://pbs.twimg.com/media/D6NAaxjWwAApBIC?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/D6NAaxjWwAApBIC?format=jpg&name=large')]",
 'inferred company': 'upmc',
 'hour': 11,
 'month': 5,
 'day of week': 'Friday',
 'Year': 2019,
 'Link': 'https://pbs.twimg.com/media/D6NAaxjWwAApBIC?format=jpg&name=small',
 'Media Type': 'Photo',
 'image_path': '0.jpg',
 'image_caption': 'arafed bee on a purple flower with yellow pollen',
 'prediction': 'upmc with username UPMCnews posted a tweet at 2019-05-10 11:44:04 with an image that is arafed bee on a purple flower with yellow pollen with number of likes 2 that reads ->: '}

In [15]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [16]:
data['test'][0]['prediction']

'upmc with username UPMCnews posted a tweet at 2019-05-10 11:44:04 with an image that is arafed bee on a purple flower with yellow pollen with number of likes 2 that reads ->: '

In [30]:
output = []

In [ ]:
for i in range(500,1000): #predictions
  txt = data['test'][i]['prediction']
  batch = tokenizer(txt , return_tensors='pt')
  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=50)
    output.append(tokenizer.decode(output_tokens[0], skip_special_tokens=True))
    print(i)

In [19]:
output

['upmc with username UPMCnews posted a tweet at 2019-05-10 11:44:04 with an image that is arafed bee on a purple flower with yellow pollen with number of likes 2 that reads ->:  UPMC is proud to be a part of the #Bees4Life campaign, which is a partnership between the <mention> and <mention>. The campaign is focused on raising awareness about the importance of pollinators and the need to protect',
 'rbc with username rbcgamnews posted a tweet at 2019-09-03 14:00:01 with an image that is a white and blue brochure with a green grass and a blue background with number of likes 1 that reads ->:  The #Gamecenter is now open! 🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮�',
 'bbc with username BBCSport posted a tweet at 2018-12-02 05:51:16 with an image that is a yellow and white sign that says breaking with number of likes 502 that reads ->:  #BreakingNews\n\n<mention> has signed a new contract with <mention> until 2022\n\n<hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyper',
 'bbc wit

In [24]:
output[499]

"bbc with username BBCSport posted a tweet at 2019-03-05 21:08:54 with an image that is the logo of tottenham football club and the goal with number of likes 326 that reads ->:  Tottenham's new logo is a tribute to the club's history and heritage. <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink> <hyperlink"

In [25]:
dicti = {'content':output}

In [26]:
dff = pd.DataFrame(dicti)

In [ ]:
for i in range(len(dff)):

  dff.at[i,'content'] = dff.at[i,'content'].split("->: ")[1]
  print(i)

In [28]:
dff

,content
0,UPMC is proud to be a part of the #Bees4Life ...
1,The #Gamecenter is now open! 🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮🎮�
2,#BreakingNews\n\n<mention> has signed a new c...
3,"#LONFC are back in action tomorrow night, wit..."
4,Congratulations to our UCLA Gymnastics Team o...
...,...
495,#PS4 is the best console ever. #E32018 <hyper...
496,UPMC Health System is proud to be a part of t...
497,<mention> <mention> <mention> <mention> <ment...
498,感谢您一直以来对我们的信任和支持，我们将继续努力，为客户提供更优质的产品和服务。 <hype...


In [29]:
dff.to_excel('output_0-500.xlsx')  #saving predictions